## Choose and split your dataset

In [12]:
import ipynb.fs #Boilerplate for ipynb notebooks as modules

from .defs.Facenet_impl import get_face_embeddings 
import tensorflow as tf
import numpy as np
import sklearn
from facenet.src import facenet
import os
import sys
import math
import imageio

/home/prudhvi/Desktop/Distillation/mobilenet-facenet-distillation/Facenet_impl.ipynb:61: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  "### Results of current model being used\n",


In [5]:
# Choose your seed
seed = 1
np.random.seed(seed=seed)
#Obtain dataset
dataset_path = "~/Desktop/Datasets/lfw/lfw_mtcnnpy_160"
dataset = facenet.get_dataset(dataset_path)

In [6]:
def split_dataset(dataset, min_nrof_images_per_class, nrof_train_images_per_class):
    train_set = []
    test_set = []
    for cls in dataset:
        paths = cls.image_paths
        # Remove classes with less than min_nrof_images_per_class
        if len(paths)>=min_nrof_images_per_class:
            np.random.shuffle(paths)
            train_set.append(facenet.ImageClass(cls.name, paths[:nrof_train_images_per_class]))
            test_set.append(facenet.ImageClass(cls.name, paths[nrof_train_images_per_class:]))
    return train_set, test_set

In [7]:
train_set, test_set = split_dataset(dataset, 0, 35)
train_paths, train_labels = facenet.get_image_paths_and_labels(train_set)
print('Number of classes: %d' % len(train_set))
print('Number of images: %d' % len(train_paths))
test_paths, test_labels = facenet.get_image_paths_and_labels(test_set)
print('Number of classes: %d' % len(test_set))
print('Number of images: %d' % len(test_paths))

Number of classes: 5749
Number of images: 12020
Number of classes: 5749
Number of images: 1213


### Get face embeddings for train/test datasets and save them to disk while doing this

In [ ]:
# Save your data to disk after splitting
train_set_emb = get_face_embeddings(train_set, save_to_disk = True, save_dir = "/home/prudhvi/Desktop/Datasets/lfw/lfw_mtcnn_160_split/train/data/")
test_set_emb = get_face_embeddings(test_set, save_to_disk = True, save_dir = "/home/prudhvi/Desktop/Datasets/lfw/lfw_mtcnn_160_split/test/data/")


In [ ]:
def create_emb_dict(paths, emb_array):
    emb_dict={}
    for i in range(len(paths)):
        image_name = paths[i].split('/')[-1]
        emb_dict[image_name] = emb_array[i]
    return emb_dict

### Create a dictionary of the face embeddings and their respective images

In [ ]:
train_set_dict = create_emb_dict(train_paths, train_set_emb)
test_set_dict = create_emb_dict(test_paths, test_set_emb)

In [ ]:
# Save the dictionaries to disk
np.save('data/face_emb_logits/train_set',train_set_dict)
np.save('data/face_emb_logits/test_set',test_set_dict)